In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [ ]:
def get_stats(h_data, h_signal):
    norm = 2*np.sqrt(np.sum(h_data.errors()**2))/h_signal.counts().sum()
    w = pyhf.simplemodels.uncorrelated_background(signal=(norm*h_signal.counts()).tolist(), bkg=h_data.counts().tolist(), bkg_uncertainty=h_data.errors().tolist())
    data = h_data.counts().tolist() + w.config.auxdata

    obs_limit, exp_limit, (scan, results) = pyhf.infer.intervals.upperlimit(
          data, w, poi, level=level, return_results=True
      )
    obs_limit, exp_limit = norm*obs_limit, [ norm*lim for lim in exp_limit ]
    return norm, w, data, obs_limit, exp_limit

In [ ]:
h_gnn = {mx : {} for mx in MX}
limit_gnn = {mx : {} for mx in MX}
limit_my_gnn = {my : {} for my in MY}

for mx in gnn_n.keys():
    for my in gnn_n[mx]:
        print(mx, my)
        v = gnn_n[mx][my]
        # print(gnn_n[mx][my])

        # h_gnn[mx][my] = up.open(filename)['signal']
        norm, w, data, obs_limit, exp_limit = get_stats(h_data_gnn, v)
        limit_gnn[mx][my] = np.array(exp_limit)*300
        limit_my_gnn[my][mx] = np.array(exp_limit)*300


In [53]:
import ROOT

# stops root from drawing shit
ROOT.gROOT.SetBatch(True)

def write_limits(mx, my):
    print('writing limits for', mx, my)
    # f =ROOT.TFile(f"~/nobackup/tmp/{mx}_{my}_limit_gnn.root","recreate")
    # f.Close()


    with ut.recreate(f"/uscms_data/d3/ekoenig/tmp/{mx}_{my}_limit_gnn.root") as f:
        dummy = np.random.uniform(size=10000)
        f['histo'] = np.histogram(dummy, bins=np.linspace(0,1,20))
        f['limits'] = {'mx':[mx], 'my':[my],'exp_limits':np.random.uniform(size=1)}
    # v = gnn_n[mx][my]
    # filename = f"{model_savein}/root_files/n_asr_{mx}_{my}.root"
    # fout = ROOT.TFile(filename,"recreate")
    # fout.cd()

    # canvas = ROOT.TCanvas('c1','c1', 600, 600)
    # canvas.SetFrameLineWidth(3)
    # canvas.Draw()

    # h_title = f"signal"
    # ROOT_hist = ROOT.TH1D(h_title,";m_{X} [GeV];Events",nbins-1,array('d',list(gnn_4b_data.mBins)))
    # for i,(val) in enumerate(v):
    #     ROOT_hist.SetBinContent(i+1, val) 

    # ROOT_hist.Draw("hist")
    # ROOT_hist.Write()
    # fout.Close()

In [54]:
mxmy = [ (mx,my) for mx, my in np.random.randint(100000,size=(5,2)) ]

In [56]:
for mx, my in tqdm(mxmy): write_limits(mx, my)

100%|██████████| 5/5 [00:00<00:00, 39.54it/s]


writing limits for 89988 67163
writing limits for 1580 75534
writing limits for 41726 48972
writing limits for 76561 95590
writing limits for 17319 94433


In [50]:
import multiprocess as mp


In [51]:
%%time 

nthreads = 10
with mp.pool.ThreadPool(nthreads) as pool:
    pool.map(lambda mxmy : write_limits(*mxmy), mxmy)

writing limits for 40085 37864
writing limits for 21213 91637
writing limits for 84160 58860
writing limits for 86947 85158
writing limits for 65581 86627
CPU times: user 70.7 ms, sys: 13.4 ms, total: 84.1 ms
Wall time: 96.3 ms


In [58]:
f = ut.open('/uscms_data/d3/ekoenig/tmp/merged_limits.root')

In [61]:
limits = f['limits'].arrays()

In [64]:
limits.mx, limits.my, limits.exp_limits

(<Array [1580, 17319, 41726, 76561, 89988] type='5 * int64'>,
 <Array [75534, 94433, 48972, 95590, 67163] type='5 * int64'>,
 <Array [[0.177], [0.428], ... [0.501], [0.686]] type='5 * 1 * float64'>)